# Open In Colab



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HelloJahid/Project-NID/blob/main/PYTESSER-OCR/PYTESSEROCR-ocr.ipynb)


# Install & Import Module

In [ ]:
# install easy-ocr module

!pip install pytesseract
!sudo apt install tesseract-ocr

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=a6d15c1cfbe3fbc9198ad5d63e5155cdddd6e4932bd28e897d11b298b6e75495
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 12 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http:/

In [ ]:
# import required module
import matplotlib.pyplot as plt
import cv2, os
from pylab import rcParams
from IPython.display import Image
import nltk
import matplotlib.pyplot as plt 
import pytesseract


import cv2
import numpy as np
import dlib
import glob

%matplotlib inline
rcParams['figure.figsize'] = 8, 16
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Download Required File

In [ ]:
# download file from github link
# this file used for detection of facial landmarks detection as well face recognition

! wget https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat

--2020-11-14 17:30:29--  https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[===================>]  95.08M   281MB/s    in 0.3s    

2020-11-14 17:30:30 (281 MB/s) - ‘shape_predictor_68_face_landmarks.dat’ saved [99693937/99693937]



# Making Folder

In [ ]:
# make a folder to store split text image from main image
if os.path.exists('/content/manual-cropedd'):
    pass
else:
    os.mkdir('/content/manual-cropedd')


# Dataset Making - Implemenation

In [ ]:
# instantiate dlib for face frontal detection land marks detection
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

def make_dataset(img_path, isDelete=True):

    # delete the pre-load images from this folder
    if isDelete:
        files = glob.glob('/content/manual-cropedd/*')
        for f in files:
            os.remove(f)


    # load image and resize for further used
    filename = img_path
    img = cv2.imread(cv2.samples.findFile(filename)) # read image   
    img = cv2.resize(img, (640, 480))  # resize image

    imgOriginal = img.copy() # make a copy the original image
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert image into grayscale

    # detect faces from image and its corresponding x-axis and y-axis location
    # cropped face from image according to x and y location
    faces = detector(imgOriginal)  # detect all faces region 
    for face in faces:
        x1,y1 = face.left(),face.top() 
        x2,y2 = face.right(),face.bottom()

        # cropped image
        cropped = imgOriginal[y1-25:y2+150 , x2+10:x2 + 320]

        # saved the croped image
        cv2.imwrite("NID_cropped.png", cropped)
    
        # Cropped ony text region using sliding window method
        img = cropped
        x_val, y_val, c = img.shape

        x , y = 0, 0
        for i in range(y_val):
            x = 0
            y = y + 10
            w, h = y_val, 60


            if y+10 > y_val:
                break

            try:
                crop_img = img[y:y+h, x:x+w]
                img_name = "/content/manual-cropedd/"+str(i)+".jpg"
                cv2.imwrite(img_name, crop_img)
            except:
                pass



# PYTESSEROCR Implementation

In [ ]:
# month name
MONTH_NAME = ['Jan', 'Feb', 'Apr' 'Mar', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']


# this function used to extract age from text
def PYTESS_OCR_month_matching(sent):
    sent =  sent.strip()
    word_token = nltk.tokenize.word_tokenize(sent)
    for i, word in enumerate(word_token):
        if word in MONTH_NAME:
            date = word_token[i-1]

            month = word_token[i]


            year = word_token[i+1]
            

            b_date = date + " " + month + " " + year
            # print(b_date)
            return b_date

#  this function used to extract NID no from text
def PYTESS_extract_digit(sent):
    nmb_list = [s for s in sent.split() if s.isdigit()]
    nmb = "".join(nmb_list)
    nmb_list, nmb, len(nmb)

    if len(nmb) > 8:
        return nmb
    else:
        return None

#  this function used to extract Name from text
def PYTESS_name_check(sent):
    sent =  sent.strip()
    upper_res = [char for char in sent if char.isupper()]
    if len(upper_res) > 8 and sent[0].isupper() and sent[1].isupper() and sent[-2].isupper() and sent[-1].isupper():
        #print("*********************************************************")
        return "ok"
    else:
        return "notok"


# This function used to extract information: Name, Age, NID number
# as well as all text detect by OCR engine
def Extract_NID_INFO_PYTESS_OCR():
    IMG_DIR = '/content/manual-cropedd'

    sent_list = list()

    name ="Not Found"
    age ="Not Found"
    NID_NUM ="Not Found"
    for img_name in os.listdir(IMG_DIR):
        img_path = IMG_DIR + "/" +img_name
        img = cv2.imread(img_path)

        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        sent = pytesseract.image_to_string(rgb)
        
        name_status = PYTESS_name_check(sent)
            
        b_date = PYTESS_OCR_month_matching(sent)
        NID_no = PYTESS_extract_digit(sent)
        
        #print("=============> ", sent)
        if name_status == "ok":
            name = sent
            # print(sent)
        if b_date != None:
            age = b_date
            # print(b_date)
        if NID_no != None and len(NID_no) == 10:
            NID_NUM = NID_no
            # print(NID_NUM)
        
        sent_list.append(sent)
        
    info_dict = dict()
    info_dict["Name"] = name.strip()
    info_dict["Date of Birth"] = age
    info_dict["NID_no"] = NID_NUM

    return info_dict, sent_list




# Making Dataset




In [ ]:
# make dataset for OCR
make_dataset('/content/111.png')

### Tesseract-OCR (OUTPUT)

In [1]:
#print information: Name, Age, NID number
info_east_ocr, all_info = Extract_NID_INFO_PYTESS_OCR()
print("*"*50)
print(info_east_ocr)
print("*"*50)


# print all text detected by OCR engine
print("\n\n")
print("*"*50)
print("Extracted text by OCR Engine(below): ")
for sent in all_info:
    print(sent.strip())
print("*"*50)